<a href="https://colab.research.google.com/github/vanowarna/audEng/blob/main/Activity%205/pitch_shifting_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch-pitch-shift
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import torch
import numpy as np
from scipy.io import wavfile
# from torch_pitch_shift import *

# Define input and output file paths
input_audio_path = '/content/drive/Shareddrives/AlgoPirates/vanowarna/audEng/pitch shifting/cropped_perfect_vocal.wav'
output_audio_path = '/content/drive/Shareddrives/AlgoPirates/vanowarna/audEng/pitch shifting/output_pitch_shifted_wavelet.wav'

# Read an audio file
sample_rate, input_waveform = wavfile.read(input_audio_path)

# Convert to tensor of shape (batch_size, channels, samples)
dtype = input_waveform.dtype
input_waveform_tensor = torch.tensor(
    [np.swapaxes(input_waveform, 0, 1)],  # (samples, channels) --> (channels, samples)
    dtype=torch.float32,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

def pitch_shift(
    input: torch.Tensor,
    shift: Union[float, Fraction],
    sample_rate: int,
    bins_per_octave: Optional[int] = 12,
    n_fft: Optional[int] = 0,
    hop_length: Optional[int] = 0,
) -> torch.Tensor:

    if not n_fft:
        n_fft = sample_rate // 64
    if not hop_length:
        hop_length = n_fft // 32
    batch_size, channels, samples = input.shape
    if not isinstance(shift, Fraction):
        shift = 2.0 ** (float(shift) / bins_per_octave)
    resampler = T.Resample(sample_rate, int(sample_rate / shift)).to(input.device)
    output = input
    output = output.reshape(batch_size * channels, samples)
    v011 = version.parse(torchaudio.__version__) >= version.parse("0.11.0")
    output = torch.stft(output, n_fft, hop_length, return_complex=v011)[None, ...]
    stretcher = T.TimeStretch(
        fixed_rate=float(1 / shift), n_freq=output.shape[2], hop_length=hop_length
    ).to(input.device)
    output = stretcher(output)
    output = torch.istft(output[0], n_fft, hop_length)
    output = resampler(output)
    del resampler, stretcher
    if output.shape[1] >= input.shape[2]:
        output = output[:, : (input.shape[2])]
    else:
        output = pad(output, pad=(0, input.shape[2] - output.shape[1], 0, 0))

    output = output.reshape(batch_size, channels, samples)
    return output

def test_pitch_shift_12_up():
    # Pitch up by 12 semitones
    shifted_up = pitch_shift(input_waveform_tensor, 12, sample_rate)
    assert shifted_up.shape == input_waveform_tensor.shape
    wavfile.write(
        output_audio_path + 'shifted_octave_+1.wav',
        sample_rate,
        np.swapaxes(shifted_up.cpu()[0].numpy(), 0, 1).astype(dtype),
    )

def test_pitch_shift_12_down():
    # Pitch down by 12 semitones
    shifted_down = pitch_shift(input_waveform_tensor, -12, sample_rate)
    assert shifted_down.shape == input_waveform_tensor.shape
    wavfile.write(
        output_audio_path + 'shifted_octave_-1.wav',
        sample_rate,
        np.swapaxes(shifted_down.cpu()[0].numpy(), 0, 1).astype(dtype),
    )

In [8]:
test_pitch_shift_12_up()
test_pitch_shift_12_down()